In [ ]:
!pip install python-telegram-bot openai feedparser nest_asyncio

In [ ]:
import os
import asyncio
import feedparser
import logging
from datetime import datetime

from telegram import Update
from telegram.ext import Application, CommandHandler, ContextTypes

import openai
from google.colab import userdata # برای دسترسی به secrets در Colab

import nest_asyncio
nest_asyncio.apply() # حل مشکل event loop در Colab

# تنظیمات لاگ
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# تنظیم کلیدهای API
try:
    OPENAI_API_KEY = userdata.get('OPENAI_TOKEN')
    TELEGRAM_TOKEN = userdata.get('TELEGRAM_TOKEN')

    if not all([OPENAI_API_KEY, TELEGRAM_TOKEN]):
        raise ValueError("لطفاً کلیدهای API را در بخش Secrets تنظیم کنید")

    openai.api_key = OPENAI_API_KEY

except Exception as e:
    logger.error(f"خطا در تنظیمات API: {e}")
    raise

# مدیریت کاربران
class UserManager:
    def __init__(self):
        self.users = {} # user_id: {'persona': '...', 'category': '...'}

    def _ensure_user_exists(self, user_id: int):
        """اگر کاربر وجود ندارد، یک رکورد خالی برای او ایجاد می‌کند"""
        if user_id not in self.users:
            self.users[user_id] = {}

    def set_persona(self, user_id: int, persona: str):
        self._ensure_user_exists(user_id)
        self.users[user_id]['persona'] = persona
        logger.info(f"کاربر {user_id} لحن را تنظیم کرد: {persona}")

    def set_category(self, user_id: int, category: str):
        self._ensure_user_exists(user_id)
        self.users[user_id]['category'] = category
        logger.info(f"کاربر {user_id} دسته را تنظیم کرد: {category}")

    def get_settings(self, user_id: int):
        """تنظیمات کاربر را برمی‌گرداند. اگر کاربر یا تنظیمات وجود نداشته باشند، دیکشنری خالی برمی‌گردد."""
        return self.users.get(user_id, {})

user_manager = UserManager()

# دریافت مقالات از arXiv
class ArticleFetcher:
    @staticmethod
    def get_articles(category: str, max_results: int = 1): # فقط یک مقاله لازم داریم
        try:
            url = f"http://export.arxiv.org/rss/{category}"
            feed = feedparser.parse(url)
            # مرتب‌سازی بر اساس تاریخ انتشار به صورت نزولی (جدیدترین اول)
            # feedparser معمولا خودش جدیدترین‌ها را اول برمی‌گرداند، اما برای اطمینان
            # entries = sorted(feed.entries, key=lambda e: getattr(e, 'published_parsed', datetime.min.timetuple()), reverse=True)

            return [
                {
                    'title': entry.title,
                    'summary': entry.summary,
                    'link': entry.link,
                    'published': getattr(entry, 'published', 'تاریخ نامشخص'),
                    # گاهی اوقات 'author' وجود دارد، گاهی 'authors'. سازگارتر می‌کنیم:
                    'authors': ', '.join(author.name for author in getattr(entry, 'authors', [{'name': 'ناشناس'}]))
                }
                for entry in feed.entries[:max_results] # قبلا 3 بود، اینجا 1 کافی است
            ] if feed.entries else []

        except Exception as e:
            logger.error(f"خطا در دریافت مقالات ({category}): {e}")
            return []

# تولید خلاصه با API openai
class ContentGenerator:
    @staticmethod
    async def generate_summary(text: str, persona: str):
        prompt = (
            f"شما در نوشتن خلاصه‌های جذاب و دقیق به سبک شخصیت‌های خاص، متخصص هستید.\n"
            f"وظیفه شما این است که متن زیر را به صورت یک خلاصه روان و فارسی، با لحن '{persona}' بنویسید:\n\n"
            f"{text}\n\n"
            "موارد مهم:\n"
            "- خلاصه بین 150 تا 550 کلمه باشد (کمی کوتاه‌تر برای خوانایی بهتر در تلگرام)\n"
            "- از اصطلاحات تخصصی به اندازه مناسب و قابل فهم استفاده کنید\n"
            "- لحن و واژگان شخصیت را به طور کامل رعایت کنید\n"
            "- ساختار منظم و جذابی داشته باشد\n"
            "مستقیماً به عنوان آن شخصیت بنویسید و خودتان را هوش مصنوعی معرفی نکنید."
        )
        try:
            client = openai.AsyncOpenAI(api_key=OPENAI_API_KEY)
            response = await client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a creative writer embodying various personas."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=1000
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            logger.error(f"خطا در تولید خلاصه با OpenAI: {e}")
            return "آها! یه مشکلی پیش اومد، نمی‌تونم الان اینو خلاصه کنم. بعداً دوباره امتحان کن یا با پشتیبانی تماس بگیر!"

# دستور /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    commands = [
        "🤖 **ربات خلاصه‌نویس مقالات علمی arXiv** 🤖",
        "",
        "سلام! من می‌تونم جدیدترین مقالات علمی از arXiv رو برات پیدا کنم و با لحنی که دوست داری خلاصه‌شون کنم.",
        "",
        "**دستورات اصلی:**",
        "🔹 `/setpersona [لحن دلخواه]` - لحن خلاصه‌ها رو انتخاب کن.",
        "   مثال: `/setpersona یک دانشمند شوخ طبع`",
        "   مثال: `/setpersona یک گزارشگر هیجان‌زده`",
        "",
        "🔹 `/setcat [کد موضوع]` - موضوع مقالات رو انتخاب کن.",
        "   مثال: `/setcat cs.AI` (هوش مصنوعی)",
        "   مثال: `/setcat cs.LG` (یادگیری ماشین)",
        "   (برای لیست کامل موضوعات، `/setcat` رو بدون آرگومان بفرست)",
        "",
        "🔹 `/getarticle` - بعد از تنظیم لحن و موضوع، جدیدترین مقاله رو با خلاصه دریافت کن.",
        "",
        "✨ اول لحن و موضوع رو تنظیم کن، بعد مقاله‌ها رو با سبک دلخواهت بخون! ✨"
    ]
    await update.message.reply_text("\n".join(commands), parse_mode='Markdown')

# دستور /setpersona
async def set_persona(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    if not context.args:
        examples = [
            "🎭 **تنظیم لحن برای خلاصه‌ها** 🎭",
            "",
            "لطفاً لحن مورد نظرت رو بعد از دستور بنویس.",
            "هر چی خلاق‌تر باشی، نتیجه جالب‌تر می‌شه!",
            "",
            "چند تا مثال:",
            "`/setpersona استاد دانشگاهی که ساده توضیح می‌دهد`",
            "`/setpersona یک فضانورد که از کشف جدیدش هیجان‌زده است`",
            "`/setpersona یک کارآگاه تیزبین که به جزئیات مقاله مشکوک است`",
            "`/setpersona نوجوانی که سعی دارد به دوستش توضیح دهد`"
        ]
        await update.message.reply_text("\n".join(examples), parse_mode='Markdown')
        return

    persona = ' '.join(context.args).strip()
    if not (3 <= len(persona) <= 70): # افزایش حداکثر طول برای توصیف بهتر لحن
        await update.message.reply_text("🧐 طول لحن باید بین 3 تا 70 کاراکتر باشه. یه کم دقیق‌تر بگو چه لحنی می‌خوای!")
        return

    user_manager.set_persona(user_id, persona)
    await update.message.reply_text(
        f"✅ عالی شد! لحن خلاصه‌هات از این به بعد اینه: **{persona}**\n\n"
        f"یادت نره با `/setcat` موضوع مقالات رو هم انتخاب کنی!",
        parse_mode='Markdown'
    )

# دستور /setcat
async def set_category(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    # لیست کامل‌تر و رایج‌تر از دسته‌بندی‌های arXiv در حوزه کامپیوتر
    # می‌توانید این لیست را از وبسایت arXiv یا منابع دیگر تکمیل کنید
    # https://arxiv.org/category_taxonomy
    arxiv_categories = {
        'cs.AI': "Artificial Intelligence (هوش مصنوعی)",
        'cs.LG': "Machine Learning (یادگیری ماشین)",
        'cs.CV': "Computer Vision and Pattern Recognition (بینایی کامپیوتر و بازشناسی الگو)",
        'cs.CL': "Computation and Language (پردازش زبان طبیعی)",
        'cs.RO': "Robotics (رباتیک)",
        'cs.NE': "Neural and Evolutionary Computing (محاسبات عصبی و تکاملی)",
        'cs.IR': "Information Retrieval (بازیابی اطلاعات)",
        'cs.SI': "Social and Information Networks (شبکه‌های اجتماعی و اطلاعاتی)",
        'cs.HC': "Human-Computer Interaction (تعامل انسان و کامپیوتر)",
        'stat.ML': "Statistics - Machine Learning (آمار - یادگیری ماشین)"
        # می‌توانید دسته‌بندی‌های بیشتری اضافه کنید
    }
    valid_categories_keys = [key.lower() for key in arxiv_categories.keys()]


    if not context.args:
        message_parts = [
            "📚 **انتخاب موضوع مقالات از arXiv** 📚",
            "",
            "لطفاً کد موضوع مورد نظرت رو بعد از دستور بنویس.",
            "مثلاً: `/setcat cs.AI`",
            "",
            "**موضوعات پیشنهادی (فقط کد را وارد کنید):**"
        ]
        for code, description in arxiv_categories.items():
            message_parts.append(f"🔸 `{code}` - {description}")
        message_parts.append("\nمی‌توانید کدهای دیگر را از سایت arXiv پیدا کنید.")
        await update.message.reply_text("\n".join(message_parts), parse_mode='Markdown')
        return

    category_arg = context.args[0].strip().lower()
    # برای اینکه کاربر بتواند cs.ai یا stat.ml وارد کند
    chosen_category_key = None
    for key in arxiv_categories.keys():
        if key.lower() == category_arg:
            chosen_category_key = key
            break

    if not chosen_category_key:
        await update.message.reply_text(
            "⚠️ این کد موضوع معتبر نیست یا در لیست من وجود نداره.\n"
            "لطفاً یکی از کدهای نمایش داده شده رو انتخاب کن یا کد دقیق رو از سایت arXiv بردار و وارد کن."
            "\nبرای دیدن لیست، دوباره `/setcat` رو خالی بفرست."
        )
        return

    user_manager.set_category(user_id, chosen_category_key) # ذخیره کد اصلی با حروف بزرگ و کوچک صحیح
    await update.message.reply_text(
        f"✅ بسیار خب! از این به بعد مقالات از موضوع **{arxiv_categories[chosen_category_key]}** (`{chosen_category_key}`) برات میاد.\n\n"
        f"اگه لحن رو هم تنظیم کردی، با `/getarticle` مقاله جدید بگیر!",
        parse_mode='Markdown'
    )

# دستور /getarticle
async def get_article(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    settings = user_manager.get_settings(user_id)

    persona = settings.get('persona')
    category = settings.get('category')

    if not persona:
        await update.message.reply_text(
            "⚠️ اول باید یه لحن انتخاب کنی! از دستور `/setpersona` استفاده کن.\n"
            "مثال: `/setpersona یک دانشمند کنجکاو`",
            parse_mode='Markdown'
        )
        return

    if not category:
        await update.message.reply_text(
            "⚠️ هنوز موضوع مقالات رو انتخاب نکردی! از دستور `/setcat` استفاده کن.\n"
            "مثال: `/setcat cs.LG`\n"
            "برای دیدن لیست موضوعات، `/setcat` رو خالی بفرست.",
            parse_mode='Markdown'
        )
        return

    await update.message.reply_text(f"🔍 دارم دنبال جدیدترین مقاله تو موضوع `{category}` با لحن «{persona}» می‌گردم... لطفاً چند لحظه صبر کن!")

    articles = ArticleFetcher.get_articles(category, max_results=1) # فقط یک مقاله جدید
    if not articles:
        await update.message.reply_text(f"🤔 متاسفانه مقاله جدیدی تو موضوع `{category}` پیدا نکردم. شاید بعداً دوباره امتحان کنی بهتر باشه.")
        return

    article = articles[0]
    # برای جلوگیری از طولانی شدن بیش از حد متن ورودی به OpenAI، خلاصه مقاله را کمی محدود می‌کنیم
    summary_input_text = article['summary']
    if len(summary_input_text.split()) > 700: # اگر خلاصه اصلی خیلی طولانی بود
        summary_input_text = ' '.join(summary_input_text.split()[:700]) + "..."


    await update.message.reply_text("📝 دارم خلاصه‌اش می‌کنم...")
    custom_summary = await ContentGenerator.generate_summary(summary_input_text, persona)

    # تبدیل تاریخ انتشار به فرمت خواناتر (اگر ممکن باشد)
    try:
        published_date = datetime.strptime(article['published'], "%Y-%m-%dT%H:%M:%SZ").strftime('%Y-%m-%d %H:%M')
    except ValueError:
        published_date = article['published'] # اگر فرمت متفاوت بود، همان اصلی را نمایش بده

    message = [
        f"📚 **{article['title']}**",
        f"👤 **نویسندگان:** {article['authors']}",
        f"📅 **تاریخ انتشار:** {published_date}",
        "",
        f"--- **خلاصه با لحن «{persona}»** ---",
        custom_summary,
        "",
        f"🔗 **لینک مقاله اصلی در arXiv:** {article['link']}"
    ]
    await update.message.reply_text("\n".join(message), parse_mode='Markdown', disable_web_page_preview=True)

# اجرای ربات
async def main():
    try:
        # اطمینان از اینکه OpenAI API key به درستی تنظیم شده
        if not openai.api_key:
            logger.error("کلید OpenAI API تنظیم نشده است. برنامه خاتمه می‌یابد.")
            return

        app = Application.builder().token(TELEGRAM_TOKEN).build()
        app.add_handler(CommandHandler("start", start))
        app.add_handler(CommandHandler("help", start)) # دستور help هم همان start را نشان دهد
        app.add_handler(CommandHandler("setpersona", set_persona))
        app.add_handler(CommandHandler("setcat", set_category))
        app.add_handler(CommandHandler("getarticle", get_article))

        logger.info("ربات با موفقیت راه‌اندازی شد و در حال اجراست...")
        await app.run_polling()
    except ValueError as ve: # خطای مربوط به عدم تنظیم کلیدها
        logger.error(f"خطای مقداردهی اولیه: {ve}")
    except Exception as e:
        logger.error(f"یک خطای بزرگ در اجرای ربات رخ داد: {e}", exc_info=True)


if __name__ == "__main__":
    try:
        # اطمینان از وجود توکن تلگرام قبل از اجرای main
        if not TELEGRAM_TOKEN:
            print("توکن تلگرام یافت نشد. لطفاً متغیر TELEGRAM_TOKEN را در Colab Secrets تنظیم کنید.")
        else:
            asyncio.run(main())
    except KeyboardInterrupt:
        logger.info("ربات توسط کاربر متوقف شد.")
    except Exception as e:
        # این بخش برای خطاهایی است که قبل از ورود به main یا در سطح asyncio.run رخ می‌دهد
        logger.error(f"یک خطای بحرانی در اسکریپت اصلی رخ داد: {e}", exc_info=True)